In [9]:
# 자동화하기
# 이어서 쓸 때 save_to_file과 append_to_file 함수에 mode의 파라미터를 'a'로 변경해보기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import csv

In [17]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome()
# 설치 폴더에 주의합니다. 

driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
# 19년 5월부터 로그인 페이지 주소가 살짝 바뀌었네요. 

time.sleep(3)


#쿠팡 검색
QUERY = '쿠팡'
search_QUERY = urlencode({'subquery': QUERY}, encoding='utf-8')
URL = 'https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query={search_QUERY}&head=&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2'

# 마지막 페이지까지 클릭
def go_to_last_page(url):
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    while True:
        try:
            # '다음 페이지' 버튼을 찾기
            next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.btn_page.btn_next")))
            next_button_text = next_button.text.strip()
            # 버튼이 비활성화 상태인지 확인
            if next_button_text == '다음 목록이 없습니다.':
                print("Reached the last page.")
                break

            # '다음 페이지' 버튼 클릭
            next_button.click()
            # 로딩 시간을 기다리기 위해 sleep 사용
            time.sleep(5)
        except TimeoutException:
            print("Reached the last page.")
            break

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.quit()
    return soup

# 마지막 페이지 번호 알아내기
def get_last_page(url):
    soup = go_to_last_page(url)
    page_list = []
    # 모든 'num_page' 클래스를 가진 span 태그 내에서 페이지 번호 추출
    for span in soup.find_all("span", class_="num_page"):
        # 'link_page' 클래스를 가진 태그(a 태그 또는 em 태그) 찾기
        page_tag = span.find(class_="link_page")
        if page_tag:
            # 페이지 번호를 리스트에 추가
            page_list.append(page_tag.get_text(strip=True))
    max_page = page_list[-1]
    print(f"총 {max_page}개의 페이지가 있습니다")
    return max_page

# 페이지 링크 모두 가져오기
def get_boards(page_num):
    boards=[]
    for page in range(page_num):
        boards.append(f"https://m.cafe.daum.net/rocksoccer/ADs2/search/all?query=%EC%BF%A0%ED%8C%A1&r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&sort=0&head=&page={page+1}")
    return boards

# 게시글 링크 가져오기
def get_posts():
    global QUERY
    global PAGES
    board_links = get_boards(PAGES)
    posts = []
    for board_link in board_links:
        req = requests.get(board_link)
        print(req.status_code)
        soup = BeautifulSoup(req.text, 'html.parser')
        tds = soup.find_all('a', {'class':'link_cafe #search_result_list'})
        for td in tds:
            if td is not None:
               posts.append(td['href'])
    print(f"총 {len(posts)}개의 글 링크를 찾았습니다")
    
    # 게시글 링크 csv로 저장
    post_file = open(f"ROCKSOCCER_{QUERY}_{PAGES}pages_inner_links.csv", mode='w', encoding='utf-8')
    writer = csv.writer(post_file)
    for post in posts:
        writer.writerow([post])
    post_file.close()
    return posts

def extract_info(url, wait_time = 3, delay_time = 1):
    max_retries = 5
    for attempt in range(max_retries):
        try:
            driver.get(url)
            html = driver.page_source
            time.sleep(delay_time)
            soup = BeautifulSoup(html,'html.parser')
           
            title = soup.find('h3', {'class': 'tit_subject'}).get_text(strip=True)
            user_id = soup.find('span', {'class':'txt_subject'}).split('|')[0].replace('작성자','').strip()
            post_time = soup.find('span', {'class':'txt_subject'}).split('|')[1].replace('작성시간','').strip()
            post_time = datetime.strptime(post_time, '%Y-%m-%d')
            post = soup.find(id='article').get_text(strip=True)
            view_cnt = soup.find('span', {'class':'txt_subject'}).split('|')[2].replace('조회수','').strip()
            reply_cnt = int(soup.find(class_="num_total").get_text(strip=True))
            reply_content = []
            if reply_cnt != 0:
                comment_button = driver.find_element(By.CLASS_NAME, 'link_all')
                comment_button.click()
                
                # 페이지 내의 코멘트들을 모두 찾자
                cmt_all = soup.body.find('ul', id = 'commentList').find_all('li')
    
                # 찾은 코멘트들을 언패킹하자
                for k in cmt_all:
                    if k.div.span.find_all('span', class_ = 'txt_bar'):
                        if k['class'] == ['reply_on']:
                            cmt_reply = True
                        else:
                            cmt_reply = False
                            
                        cmt_writer = k.div.find('span', class_='sr_only').next_sibling
                        cmt_time = k.div.span.find('span', class_='num_info').get_text()
                        cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                        reply_content.append({'author': cmt_writer, 'text': cmt_txt, 'time' : cmt_time})
            data = {'title': title, 'user_id': user_id, 'post_time': post_time, 'post': post, 'view_cnt': view_cnt, 'reply_cnt': reply_cnt, 'reply_content': reply_content}
            
            # 댓글이 있다면 딕셔너리에 추가
            if reply_cnt != 0:
                for i, reply_info in enumerate(reply_content):
                    data[f'reply_{i+1}_author'] = reply_info['author']
                    data[f'reply_{i+1}_text'] = reply_info['text']
                    data[f'reply_{i+1}_time'] = reply_info['time']
            print(url, '완료')
            break  # 성공적으로 크롤링이 완료되면 루프 종료
        except Exception as e:
            print(f"Crawling failed, retrying... ({attempt+1}/{max_retries})")
            time.sleep(2)  # 2초 대기 후 재시도
    else:
        print("Crawling failed after maximum retries.")
    
    return data

def get_contents():
    global rocksoccer_results
    post_links = get_posts()
    for post_link in post_links:
        content = extract_info(post_link)
        append_to_file(f"RockSoccer_{QUERY}_{PAGES}.csv", content)
    return print("모든 작업이 완료되었습니다.")

def save_to_file():
    file = open(f'RockSoccer_{QUERY}_.csv', mode='a', encoding='utf-8')
    writer = csv.writer(file)
    writer.writerow(['title', 'user_id', 'post_time', 'post', 'view_cnt', 'reply_cnt', 'reply_author', 'reply_text', 'reply_time'])
    file.close()
    return file

def append_to_file(file_name, data):
    file = open(file_name, mode='a', encoding='utf-8', newline='')  # newline 추가
    writer = csv.writer(file)

    # Write main post data to CSV
    main_post_row = [
        data['title'],
        data['user_id'],
        data['post_time'],
        data['post'],
        data['view_cnt'],
        data['reply_cnt'],
        data['reply_author'],
        data['reply_text'],
        data['reply_time']
    ]

    # Check if there are replies
    if 'reply_content' in data and data['reply_cnt'] != 0:
        # Write main post data along with reply data to CSV
        for i, reply_info in enumerate(data['reply_content']):
            writer.writerow(main_post_row + [
                reply_info.get('author', ''),  # Reply author
                reply_info.get('text', ''),  # Reply text
                reply_info.get('time','')
            ])
    else:
        # Write only main post data to CSV
        writer.writerow(main_post_row + ['']*3)

    file.close()
    return

# 크롤링할 페이지 수 설정
PAGES = int(get_last_page(URL))
# 게시글 링크 수집
rocksoccer_results = save_to_file()
# 게시글 정보 크롤링 및 저장
get_contents()


Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36
Reached the last page.
총 32개의 페이지가 있습니다
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
총 636개의 글 링크를 찾았습니다
Crawling failed, retrying... (1/5)
Crawling failed, retrying... (2/5)
Crawling failed, retrying... (3/5)
Crawling failed, retrying... (4/5)
Crawling failed, retrying... (5/5)
Crawling failed after maximum retries.


KeyError: 'title'